# Cost Results Analysis

This notebook allows you to upload exported model results for visualization and analysis.

Upload a model package file downloaded from the Exports section of the cost notebook.

In [ ]:
import sys
!{sys.executable} -m pip install -r ../requirements.txt

In [ ]:
sys.path.append('../')

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import json
import binascii
import pickle
from io import StringIO
import base64

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

from ipywidgets import FileUpload, Output, Label, VBox, HBox, HTML, Layout, GridBox

from giga.viz.notebooks.components.html.sections import section
from giga.viz.notebooks.components.widgets.giga_file_upload import GigaFileUpload
from giga.viz.notebooks.components.widgets.giga_buttons import make_run_button, make_run_again_button, make_show_maps_button, make_show_full_table_button, make_rounded_button
from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput
from giga.viz.notebooks.components.dashboard.result_dashboard import ResultDashboard
from giga.viz.notebooks.maps import show_cost_map, show_electricity_map
from giga.viz.notebooks.fiber import plot_fiber_connections, default_map
from giga.data.space.model_data_space import ModelDataSpace
from giga.data.stats.result_stats import ResultStats
from giga.schemas.output import OutputSpace

upload_package = GigaFileUpload(accept='.pkl', multiple=False, description="Results Package (.pkl)")
inputs = CostEstimationParameterInput(local_data_workspace='workspace', show_map=True)


upload_output = Output()
parameter_output = Output()
results_output = Output()
map_output = Output()
table_output = Output()

data_space: ModelDataSpace = None
output_space: OutputSpace = None
results_table = None

# Capture output plots for export, so they aren't created multiple
# times, which causes memory issues
class VisualPlots():
    electricity = None
    cost = None
    fiber = None
    dashboard: ResultDashboard = None

    def populate(self, dashboard: ResultDashboard, data_space_selected: ModelDataSpace):
        self.dashboard = dashboard
        center = inputs.defaults[inputs.data_parameters().school_data_conf.country_id].data.country_center_tuple
        
        # Set up plotly maps
        if self.electricity is None:
            self.electricity = show_electricity_map(data_space_selected, location=center)
        if self.cost is None:
            self.cost = show_cost_map(data_space_selected, output_space, location=center)

        # Set up folium maps
        m = default_map(location=center)
        if self.fiber is None and output_space.fiber_costs is not None:
            self.fiber = plot_fiber_connections(data_space_selected.fiber_coordinates, data_space_selected.school_coordinates, 
                                            output_space.fiber_costs.technology_results.distances, m=m)

    def get_all(self):
        return self.dashboard.get_visual_plots() + [
            self.electricity,
            self.cost
        ] + ([self.fiber] if self.fiber is not None else [])

all_output_maps = VisualPlots()

def show_all_maps(event):
    map_output.clear_output(wait=True)
    with map_output:
        if inputs.data_parameters().school_data_conf.country_id == 'brazil':
            display(HTML("<hr><b><font color='#5b8ff0'>Unable to display maps for Brazil</b><br><br>"))
        display(section("Electricity Map", HTML(all_output_maps.electricity._repr_html_()).add_class("folium-map")))
        display(section("Cost Map", HTML(all_output_maps.cost._repr_html_()).add_class("folium-map")))

        if output_space.fiber_costs is not None:
            fiber_html = HTML(all_output_maps.fiber._repr_html_())
            fiber_html.add_class('folium-map')
            display(section("Fiber Map", fiber_html))

def show_full_table(event):
    with table_output:
        table = HTML(results_table.to_html(col_space=5, border=0))
        display(section("Cost Table", table))

def update_results_from_selection(ev=None):
    global results_table
    global data_space
    global output_space
    global inputs
    results_output.clear_output()
    with results_output:
        school_ids = inputs.get_selected_schools()
        # Reconstruct a data space
        data_space = ModelDataSpace(inputs.data_parameters())
        data_space_selected = data_space.filter_schools(school_ids)
        
        # Reconstruct an output space
        output_space_selected = output_space.filter_schools(school_ids)
        
        # Construct a results dashboard
        stats = ResultStats(data_space_selected, output_space_selected, inputs.all_tech_config())
        dashboard = ResultDashboard(stats)
        n_opex_years = inputs.scenario_parameters().years_opex

        # Populate and display
        results_table = data_space_selected.school_outputs_to_frame(output_space_selected.full_results_table(n_opex_years))
        display(HTML('<br/><br/>'))
        dashboard.populate_outputs()
        all_output_maps.populate(dashboard,data_space_selected)
        dashboard.display()
        show_maps_button = make_show_maps_button(show_all_maps)
        show_maps_button.on_click(show_all_maps)
        full_table_button = make_show_full_table_button(show_full_table)
        full_table_button.on_click(show_full_table)
        display(VBox([
            show_maps_button,
            full_table_button
        ]))

update_selection = make_rounded_button(update_results_from_selection,
                                       "Update Results")

def on_finalize_upload_click(ev):
    global results_table
    global data_space
    global output_space
    global inputs
    global results_output
    global map_output
    global table_output
    
    upload_output.clear_output()
    results_output.clear_output()
    map_output.clear_output()
    table_output.clear_output()
    with upload_output:
        if len(upload_package.value) != 1:
            print("Please upload a model package file (max 15MB)")
            return
    upload_output.clear_output()
    with upload_output:
        display(HTML("Uploading results package..."))
        # Load config and output space from pickle package
        decoded_data = binascii.unhexlify(upload_package.value[0].content.tobytes().decode('utf-8'))
        upload_output.clear_output()
        display(HTML("Loading configuration..."))
        with parameter_output:
            pkg = pickle.loads(decoded_data)
            inputs.update(pkg.config)
            inputs.set_selected_schools(pkg.selected_schools)
            inputs.freeze()
            display(inputs.parameter_input())
            display(update_selection)
        upload_output.clear_output()
        display(HTML("Loading results..."))
        output_space = pkg.output_space
    update_results_from_selection()
    upload_output.clear_output()

finalize_upload = make_rounded_button(on_finalize_upload_click, "Upload")

display(section(
    "Upload Configuration & Results",
    VBox([
        upload_package,
        finalize_upload,
        upload_output
    ])
))
display(parameter_output)
display(results_output)
display(map_output)
display(table_output)
